# 1. install libaries

In [20]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install XBRL

Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install oauth2

     |████████████████████████████████| 102kB 8.1MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# 2. import libraries

In [3]:
import requests

In [4]:
import os, re, sys, json

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [7]:
import getpass

# 3. Authentication for xbrl token

### 3.1. acess token

In [8]:
"""
get access token from XBRL API
"""
# print('Enter XBRL US Web account email: ')
email = 'jesse.c.jia@gmail.com'
# print('Account password: ')
password = 'QAZwsx123'
# print('Client ID: ')
clientid = 'bf3ba33f-e98b-4159-aea2-466445f9621f'
# print('Client secret: ')
secret = 'a3f87777-0086-4208-8d3b-df494408271e'

body_auth = {'username' : ''.join(email), 
            'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb' }
endpoint_auth = 'https://api.xbrl.us/oauth2/token'
res = requests.post(endpoint_auth, data=body_auth)
auth_json = res.json()

if 'error' in auth_json:
        print ("\n\nThere was a problem generating an access token with these credentials. Run the first cell again to enter credentials.")
else:
        print ("\n\nYour access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. \n\nFor now, skip ahead to the section 'Make a Query'.")

access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''
    
    



Your access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. 

For now, skip ahead to the section 'Make a Query'.



### 3.2. refresh token

In [9]:

"""
reconnect to XBRL API when expire
"""
token = token if newrefresh != '' else refresh_token 

refresh_auth = {'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'grant_type' : 'refresh_token', 
            'platform' : 'ipynb', 
            'refresh_token' : ''.join(token) }
refreshres = requests.post(endpoint_auth, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
print(access_token)

Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.
713878b6-d822-492f-8079-471ada20d519


# 4. Prepare Data

### 4.1 file and data path

In [10]:
"""
get document path
"""
root_path = os.path.abspath(os.path.dirname(os.getcwd()))
data_path = os.path.join(root_path, 'data')
company_path = os.path.join(data_path, 'company')
company_cik_path = os.path.join(company_path, 'CIK_Summary.csv')
sec_cfp_path = os.path.join(data_path, "SEC_FIN_data")
sec_argriculture = os.path.join(sec_cfp_path, "agriculture")

### 4.2. read cik file

In [11]:
"""
import company-cik file
"""

#read cik.csv as a pandas DataFrame
ciksDF = pd.read_csv(company_cik_path, header=None)

#set the column with all the tickers as the index
ciksDF = ciksDF.set_index(0)


### 4.3. define company name and cik converting function

In [12]:
#function to return CIK of a ticker
def getCIK(ticker):
    """
    input ticker
    return respect cik
    """
    try:
        return int(ciksDF.loc[ticker.lower(), 1])

    #return None there's no CIK for the ticker
    except:
        print(f"{ticker} does not exist.")
        return None
    
    
#function to return CIK of a list of tickers
def outputcik(TICKERS): 
    """
    input ticker list
    return cik repsectively in list
    """
    cikDir = []
    for ticker in TICKERS:
        a = str(getCIK(ticker))
        if a != 'None':
            cik = a.zfill(10)
            cikDir.append(cik)
        else:
            pass
    return cikDir

In [13]:
# test code
lis = outputcik(["uytuyfd","a"])
lis

uytuyfd does not exist.


['0001090872']

### 4.4. Read Company List

In [14]:
# company list

company_list = ["YTEN", "UAN", "CGA", "RKDA", "FMC", "MOS", "SEED", "CTVA", "MBII", "AVD", "ICL", "IPI", "NTR", "MGPI", "SMG", "CF"]

In [15]:
len(company_list)

16

### 4.5. convert company_name into cik

In [16]:
# generate cik list
cik_list = outputcik(company_list)

In [17]:
# cik_list1 = cik_list.pop(2)

In [18]:
len(cik_list)

16

In [19]:
cik_list

['0001121702',
 '0001425292',
 '0000857949',
 '0001469443',
 '0000037785',
 '0001285785',
 '0001321851',
 '0001755672',
 '0001441693',
 '0000005981',
 '0000941221',
 '0001421461',
 '0001725964',
 '0000835011',
 '0000825542',
 '0001324404']

### 4.6. financial data conceptual name

In [20]:
# financial data with the conceptual name on SEC website respectively
Financial_data_dic = {
    "revenue" : "RevenueFromContractWithCustomerExcludingAssessedTax",
    "net_income" : "NetIncomeLoss",
    "operating_income" : "OperatingIncomeLoss", 
    "asset" : "Assets",
    "current_asset" : "AssetsCurrent",
    "inventory" : "InventoryNet",
    "current_liability" : "LiabilitiesCurrent",
    "debt" : "LongTermDebtNoncurrent",
    "equity" : "StockholdersEquity",
    "eps" : "EarningsPerShareBasic"
}

### 4.7. read cfp file function

In [21]:
def read_fin_data_file(path):
    df = pd.read_csv(path)
    df = df.set_index('Unnamed: 0')
    return df

# 5. Request data via XBRL

In [22]:
def acquire_data(company_cik, financial_data):
    """
    input company cik value and financial conceptual name on SEC
    return data frame of financial value, year, entity name, and so on, (details see fiels list)
    """
    
    # Define the fields to be returned
    fields = ['period.fiscal-year.sort(DESC)',
              'period.end.sort(DESC)',
              'entity.name.sort(ASC)',
              'entity.cik.sort(ASC)',
              'concept.local-name.sort(ASC)',
              'fact.value',
              'unit',
              'report.filing-date']
    
    # Filter the query for specific XBRL elements (tags) to research
    XBRL_Elements = [financial_data
                ]
    
    # Filter companies for the query
    companies_cik = [company_cik
                ]
    
    # api parameters
    params = {'fields': ','.join(fields),
         'concept.local-name': ','.join(XBRL_Elements),
         'entity.cik': ','.join(companies_cik),
         #'period.fiscal-year': ','.join(years),
         'period.fiscal-period': 'Y',
         'fact.has-dimensions': 'FALSE',
         'fact.ultimus': 'TRUE'
         }
    
    search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
    res_json = res.json()
    res_df = pd.DataFrame(res_json['data'])
    
#     print(res_json['paging'])
#     print(res_df)
    return res_df

In [23]:
# TEST CODE
df_test = acquire_data('0001121702',"Assets")

In [24]:
df_test

,period.fiscal-year,period.end,entity.name,entity.cik,concept.local-name,fact.value,unit,report.filing-date
0,2020,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,14522000,USD,2021-08-11
1,2019,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,16718000,USD,2021-03-16
2,2018,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,12960000,USD,2020-03-25
3,2017,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,16882000,USD,2019-03-28
4,2016,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,10748000,USD,2018-03-12
5,2015,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,17088000,USD,2017-03-30
6,2014,None,"METABOLIX, INC.",0001121702,Assets,23135000,USD,2016-03-29
7,2013,None,"METABOLIX, INC.",0001121702,Assets,26738000,USD,2015-03-25
8,2012,None,"METABOLIX, INC.",0001121702,Assets,53510000,USD,2014-03-28
9,2011,None,"METABOLIX, INC.",0001121702,Assets,82912000,USD,2013-03-28


In [25]:
# for cik in cik_list:
#     #print(cik)
#     cik_fin_value_df = df_empty = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
#     for k, v in Financial_data_dic.items():
#         print(cik,v)
#         temp_df = acquire_data(cik, v) asset
#         print(temp_df)
        




In [43]:
def generate_financial_data(cik, Financial_data_dic):
    """
    input company's cik value and financial item dic
    acquire financial value through xrbl API,
    calculate financial ratio and append in data frame
    return a data frame with year, company name, financial ratios
    """
    
    # create a empty data frame for use
    cik_fin_value_df = pd.DataFrame()
    # cik = "0001121702"
    for k, v in Financial_data_dic.items():
        print(f"current cik is {cik}, acquire {k}")
        
        # return first financial values into df
        if k == "revenue":
            
            cik_fin_value_df = acquire_data(cik, v)
            if cik_fin_value_df.empty:
                pass
            else:
                cik_fin_value_df.rename(columns = {"fact.value" : k}, inplace = True)
                cik_fin_value_df[k] = pd.to_numeric(cik_fin_value_df[k])
                cik_fin_value_df.drop(labels = "concept.local-name", axis = 1, inplace = True)
                cik_fin_value_df.drop(labels = "unit", axis = 1, inplace = True)
                cik_fin_value_df.drop(labels = "report.filing-date", axis = 1, inplace = True)

    #         cik_fin_value_df.drop(labels = "entity.name", axis = 1, inplace = True)

        
        # create temp df, and acquire rest financial data, then concate with inital df
        else:
#             print(cik,k,v)
            temp_df = acquire_data(cik, v)
            if temp_df.empty:
                temp_df[k] = np.NaN
                cik_fin_value_df = pd.concat([cik_fin_value_df, temp_df], axis = 1)
            else:
                temp_df.rename(columns = {"fact.value" : k}, inplace = True)
                temp_df[k] = pd.to_numeric(temp_df[k])

                temp_df.drop(labels = "concept.local-name", axis = 1, inplace = True)
    #             temp_df.drop(labels = "period.end", axis = 1, inplace = True)
                temp_df.drop(labels = "unit", axis = 1, inplace = True)
                temp_df.drop(labels = "report.filing-date", axis = 1, inplace = True)
    #             temp_df.drop(labels = "entity.name", axis = 1, inplace = True)


    #             if len(temp_df["period.end"]) >= len(cik_fin_value_df["period.end"]):
    #                 cik_fin_value_df.drop(labels = "period.end", axis = 1, inplace = True)
    #             else:
                temp_df.drop(labels = "period.end", axis = 1, inplace = True)
    
                # keep the most company name column
                try:
                    if len(temp_df["entity.cik"]) >= len(cik_fin_value_df["entity.cik"]):
                        cik_fin_value_df.drop(labels = "entity.name", axis = 1, inplace = True)
                        cik_fin_value_df.drop(labels = "entity.cik", axis = 1, inplace = True)
                    else:
                        temp_df.drop(labels = "entity.name", axis = 1, inplace = True)
                        temp_df.drop(labels = "entity.cik", axis = 1, inplace = True)
                except:
                    pass



    #             cik_fin_value_df = pd.concat([cik_fin_value_df, temp_df], axis = 1)
    #             cikabs_fin_value_df = pd.concat((cik_fin_value_df, temp_df), axis = 1, join = "outer")
                try:
                    cik_fin_value_df = pd.merge(cik_fin_value_df, temp_df, on = "period.fiscal-year", how = "outer")
                except:
                    pass
    
    # calculate financial ratios
#     Ratios_calculator(cik_fin_value_df)
    print(f"{cik} fin data acquire compeleted.")
    save_csv(cik_fin_value_df, sec_argriculture, cik)
    
    return cik_fin_value_df



# cik_fin_value_df["ROA1"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["asset"], axis = 1)
# cik_fin_value_df["ROE1"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["equity"], axis = 1)
# cik_fin_value_df["ROC1"] = cik_fin_value_df.apply(lambda x:(x["net_income"] + x["debt"])/x["asset"], axis = 1)
# cik_fin_value_df["OperatingMargin"] = cik_fin_value_df.apply(lambda x:x["operating_income"]/x["revenue"], axis = 1)
# cik_fin_value_df["NetMargin"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["revenue"], axis = 1)
# cik_fin_value_df["CurrentRatio"] = cik_fin_value_df.apply(lambda x:x["current_asset"]/x["current_liability"], axis = 1)
# cik_fin_value_df["QuickRatio"] = cik_fin_value_df.apply(lambda x:(x["current_asset"]-x["inventory"])/x["current_liability"], axis = 1)
# cik_fin_value_df["DERatio"] = cik_fin_value_df.apply(lambda x:x["debt"]/x["equity"], axis = 1)



In [27]:
# TEST CODE

# test1_df = generate_financial_data('0000857949', Financial_data_dic)

In [28]:
test1_df

NameError: name 'test1_df' is not defined

In [82]:
def ROA1(df):
    df["ROA1"] = df.apply(lambda x:x["net_income"]/x["asset"], axis = 1)

def ROE1(df):
    df["ROE1"] = df.apply(lambda x:x["net_income"]/x["equity"], axis = 1)

def ROC1(df):
    df["ROC1"] = df.apply(lambda x:(x["net_income"] + x["debt"])/x["asset"], axis = 1)

def Op_Margin(df):
    df["OperatingMargin"] = df.apply(lambda x:x["operating_income"]/x["revenue"], axis = 1)
    
def Net_Margin(df):
    df["NetMargin"] = df.apply(lambda x:x["net_income"]/x["revenue"], axis = 1)
    
def Current_Ratio(df):
    df["CurrentRatio"] = df.apply(lambda x:x["current_asset"]/x["current_liability"], axis = 1)

def Quick_Ratio(df):
    df["QuickRatio"] = df.apply(lambda x:(x["current_asset"]-x["inventory"])/x["current_liability"], axis = 1)

def DE(df):
    df["DERatio"] = df.apply(lambda x:x["debt"]/x["equity"], axis = 1)


In [94]:
def Ratios_calculator(df):
    
    for k, v in Financial_data_dic.items():
        df[k] = pd.to_numeric(df[k])
    ROA1(df)
    ROE1(df)
    ROC1(df)
    Op_Margin(df)
    Net_Margin(df)
    Current_Ratio(df)
    Quick_Ratio(df)
    DE(df)
    
    return df

In [93]:
def generate_financial_ratio(cik, industry, valid_list, invalid_list):
#     generate_financial_data(cik, fin_data_dic)

    df = pd.DataFrame()
    try:
        df = read_fin_data_file(f'{sec_cfp_path}/{industry}_comprehensive/{cik}.csv')
        print(1)
        Ratios_calculator(df)
        print(f"{cik} fin ratios calculated")
        valid_list.append(cik)
    except:
        print(f"{cik} has no ratios return")
        invalid_list.append(cik)
    


#     try:
#         Ratios_calculator(df)
#         #print(df)
#         print(f"{cik} fin ratios calculated")
#         valid_list.append(cik)
#     except:
#         print(f"{cik} has no ratios return")
#         invalid_list.append(cik)
#         pass
    
    return df, valid_list, invalid_list

In [71]:
def output_fin_data_file(cik_list, fin_data_dic, industry):
    
    valid_company_list = []
    invalid_company_list = []
    path = os.path.join(sec_cfp_path, industry)

    for c in cik_list:
        df = generate_financial_data(c, fin_data_dic)
        print("df is here")
        print(df)
        save_csv(df, path, c)
    
    print(f"The list cik companies have connection with SEC website:")
    for cik in valid_company_list:
        print(cik)

    print(f"The list cik companies have no connection with SEC website, no return data via XBRL:")
    for cik in invalid_company_list:
        print(cik)
    
    return None

In [87]:
def output_file(cik_list, fin_data_dic, industry):
    industry_df = pd.DataFrame()
    
    valid_company_list = []
    invalid_company_list = []

    for c in cik_list:
#         generate_financial_data(c, fin_data_dic)
        temp_df, valid_company_list, invalid_company_list = generate_financial_ratio(c, industry, valid_company_list, invalid_company_list)
        
        industry_df = pd.concat([industry_df, temp_df], axis = 0)
        
#         print(f"{c} has been added.")
    
    industry_df.to_csv(f"{sec_cfp_path}/{industry}.csv")
    
#     print(f"The list cik companies have connection with SEC website: {valid_company_list}.")
#     print(f"The list cik companies have no connection with SEC website, no value returned by using XBEL: {valid_company_list}.")
    
    return industry_df, valid_company_list, invalid_company_list

# 6. output process

### 6.1. define save function

In [41]:
def save_csv(file, path, industry):
    file.to_csv(f"{path}/{industry}.csv")
    print(f"{file} has been saved as csv.")
    return None

### 6.2. output financial data into single file for each compnay

In [77]:
output_fin_data_file(cik_list, Financial_data_dic, "agriculture")

current cik is 0001121702, acquire revenue


KeyError: 'data'

### 6.3. generate cfp in one file|

In [89]:
indus, valid_company_list, invalid_company_list = output_file(cik_list, Financial_data_dic, "agriculture")

1
Unnamed: 0
0.0       799000.0
1.0       806000.0
2.0       556000.0
3.0       944000.0
4.0      1159000.0
5.0      1350000.0
6.0      2800000.0
7.0      5394000.0
8.0     42316000.0
9.0      1425000.0
10.0      448000.0
11.0     1425000.0
12.0     1555000.0
13.0     1683000.0
14.0     4590000.0
15.0     2781000.0
NaN            NaN
Name: revenue, dtype: float64 0001121702
Unnamed: 0
0.0    -10206000.0
1.0    -12956000.0
2.0     -9185000.0
3.0     -9396000.0
4.0     -7604000.0
5.0    -23681000.0
6.0    -29534000.0
7.0    -30506000.0
8.0      3630000.0
9.0    -38785000.0
10.0   -38803000.0
11.0   -37957000.0
12.0   -36005000.0
13.0   -27875000.0
14.0   -16062000.0
15.0    -7625000.0
NaN            NaN
Name: net_income, dtype: float64 0001121702
Unnamed: 0
0.0     -9609000.0
1.0     -8596000.0
2.0     -9319000.0
3.0     -9283000.0
4.0    -10248000.0
5.0    -12469000.0
6.0    -26829000.0
7.0    -28540000.0
8.0      3929000.0
9.0    -38861000.0
10.0   -38939000.0
11.0   -38729000.0
12.0  

/Users/j/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


1
Unnamed: 0
0     395521000
1     362745000
2     376089000
3     347448000
4     318263000
5     327604000
6     313403000
7     323264000
8     334335000
9     146563000
10    248189000
11    201971000
12    291812000
Name: revenue, dtype: int64 0000835011
Unnamed: 0
0     40345000
1     38793000
2     37284000
3     41823000
4     31184000
5     26191000
6     23675000
7     -4929000
8      1624000
9     10635000
10    -1313000
11     8738000
12   -69123000
Name: net_income, dtype: int64 0000835011
Unnamed: 0
0     54241000
1     47242000
2     50148000
3     42909000
4     41975000
5     32850000
6     16619000
7     -5199000
8      -944000
9      3162000
10      645000
11     7255000
12   -79008000
Name: operating_income, dtype: int64 0000835011
Unnamed: 0
0     366575000
1     322597000
2     277892000
3     240328000
4     225336000
5     194310000
6     160215000
7     151329000
8     163171000
9     169157000
10    136483000
11    121284000
12    145132000
Name: asset, dtype:

In [39]:
print(f"The list cik companies have connection with SEC website:")
for cik in valid_company_list:
    print(cik)


The list cik companies have connection with SEC website:
0001121702
0001425292
0001469443
0000037785
0001285785
0001441693
0000005981
0000835011
0000825542


In [40]:
print(f"The list cik companies have no connection with SEC website, no return data via XBRL:")
for cik in invalid_company_list:
    print(cik)

The list cik companies have no connection with SEC website, no return data via XBRL:
0000857949
0001321851
0001755672
0000941221
0001421461
0001725964
0001324404


In [90]:
df = read_fin_data_file(f'{sec_cfp_path}/agriculture_comprehensive/0001121702.csv')

In [95]:
dfr = Ratios_calculator(df)

ValueError: Unable to parse string "109,588,000" at position 14